# 16. Generating Forms with [WTForms](http://wtforms.readthedocs.org/en/latest/index.html)

In the previous notebooks, we learned how to use Flask to create functions or classes that map to URLs. We also learned that there were several ways to send variables to those functions. In the last notebook, we learned about creating HTML templates using Jinja2 that the server would send in response to requests. This notebook will focus on creating forms using WTForms.

With enough practice, you'll find out that when rendering forms and receiving form data, input types should align with your data models and that valdation could be done on both client and server. You'll also discover that there are patters and that you may be repeating a lot of code.

WTForms lets you define classes that will generate the HTML form that handles input validation without boilerplate or repetitive code. When `wtforms` is loaded in a template, a form object becomes accessible with defined fields that map to your data. To process a submitted form, just instantiate the form, passing the`request.POST` data to it as parameters. The `wtforms.Form` object has a `validate()` method that takes care of validation for you. If the form is valid, you can process the data as needed. If not, it also has built-in error messages that you can display.

That was quite a long explanation. Let's see some code.

## Install WTForms

In [ ]:
!pip install wtforms
# !pip install flask-wtf

## Create a Form class

In [ ]:
# our form class

from wtforms import Form, PasswordField, TextField, validators
# from flask.ext.wtf import Form

class RegistrationForm(Form):
    username = TextField('Username', [validators.Length(min=4, max=25)])
    email = TextField('Email Address', [validators.Length(min=6, max=35)])
    password = PasswordField('New Password', [
        validators.Required(),
        validators.EqualTo('confirm', message='Passwords must match')
    ])
    confirm = PasswordField('Repeat Password')

In the `RegistrationForm` class defined above, the `Form` and some Field objects, as well as the `wtforms.validator` module were imported.

Our `RegistrationForm` class derives from the `wtforms.Form` object. Its attributes are instances of `wtforms.BooleanField` and `wtforms.StringField` objects. When each of these fields were instantiated, a list of `validators` were passed.

There is a Flask extension called Flask-WTForms that provides some additional helper methods like Form.validate_on_submit(). It's a separate package that can be installed with `pip install flask-wtf`.

## Create a view function or class

In [ ]:
# our view function
from flask import Flask, render_template, request

@app.route('/', methods=['GET', 'POST'])
def register():
    form = RegistrationForm(request.form or None)
    context = {
        'form': form,
    }

    # if request.method == 'POST' and form.validate():
    if form.validate_on_submit():  # this method is available in `from flask.ext.wtf.Form`
        context['form_data'] = form.data

    return render_template('register.html', **context)

if __name__ == '__main__':
    app.secret_key = 'SECRET_KEY'
    app.run(debug=True)

In our `register()` function, the `RegistrationForm` gets instantiated either with the `request.form` data if it is available or `None` as default. Validation will fail when there is no data but will go through the validation process if there is. Form errors will be returned if validation fails.

In our conditional statement, the form data is added to the context data passed to the template. We'll just render this in the page. The template code is below.

## Render the {{ form }}

In [ ]:
%%html
<h1>Registration</h1>
{% if form_data %}
    <h2>Thanks for submitting!</h2>
    {% for key, value in form_data.items() %}
        <p>{{ key }}: {{ value }}</p>
    {% endfor %}
{% endif %}

<form method="POST" action="/">
    <div>{{ form.username.label }}: {{ form.username() }}</div>
    {% if form.username.errors %}
        <ul class="errors">{% for error in form.username.errors %}<li>{{ error }}</li>{% endfor %}</ul>
    {% endif %}

    <div>{{ form.email.label }}: {{ form.email() }}</div>
    {% if form.email.errors %}
        <ul class="errors">{% for error in form.email.errors %}<li>{{ error }}</li>{% endfor %}</ul>
    {% endif %}

    <div>{{ form.password.label }}: {{ form.password() }}</div>
    {% if form.password.errors %}
        <ul class="errors">{% for error in form.password.errors %}<li>{{ error }}</li>{% endfor %}</ul>
    {% endif %}

    <div>{{ form.confirm.label }}: {{ form.confirm() }}</div>
    {% if form.confirm.errors %}
        <ul class="errors">{% for error in form.confirm.errors %}<li>{{ error }}</li>{% endfor %}</ul>
    {% endif %}

    {{ form.csrf_token }}
    <button value="submit">Submit</button>
</form>

In our template code, the first part just displays the submitted form data if available.

The second part is our form. The `<form>` tag should be included in the HTML together with the `method` and `action` attributes.

Nested in the HTML form are attributes of the form object generated from WTForms. We are displaying each per form field:
- Label
- Input
- Errors (if any)

Then finally the submit button.

Let's see the code in action.

In [ ]:
!python app.py

This small Flask demo uses WTForms to create a registration form that displays submitted data on the page. It uses the code in the examples. Read the documentation for more information.

#### Exercise:

In the previous notebook, you created a webpage that showcases an upcoming product. Recreate it using WTForms. Add an admin login to it. You can hardcode the admin credentials for now.